<a href="https://colab.research.google.com/github/dude123studios/AdvancedDeepLearning/blob/main/Vanilla_AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as K
import matplotlib.pyplot as plt

In [2]:
np.random.seed(11)
tf.random.set_seed(11)
batch_size = 256
epochs = 50
learning_rate = 1e-3
momentum = 8e-1
hidden_dim = 128
original_dim = 784

In [3]:
(x_train, _), (x_test, _) = K.datasets.mnist.load_data()
x_train = x_train / 255.
x_test = x_test / 255.
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1]*x_train.shape[2]))
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1]*x_test.shape[2]))
train_data = tf.data.Dataset.from_tensor_slices(x_train).batch(batch_size)

In [4]:
class Encoder(K.layers.Layer):
  def __init__(self, num_units):
    super(Encoder, self).__init__(num_units)
    self.dense = K.layers.Dense(num_units, activation='relu')
  def call(self, x):
    x = self.dense(x)
    return x

In [5]:
class Decoder(K.layers.Layer):
  def __init__(self, num_units):
    super(Decoder, self).__init__(num_units)
    self.dense = K.layers.Dense(num_units, activation='relu')
  def call(self, x):
    x = self.dense(x)
    return x

In [6]:
class Autoencoder(K.Model):
  def __init__(self, hidden_dim, original_dim):
    super(Autoencoder, self).__init__()
    self.loss = []
    self.encoder = Encoder(hidden_dim)
    self.decoder = Decoder(original_dim)

  def call(self, input_features):
    encoded = self.encoder(input_features)
    reconstructed = self.decoder(encoded)
    return reconstructed

In [7]:
autoencoder = Autoencoder(hidden_dim, original_dim)
opt = tf.keras.optimizers.Adam(learning_rate=1e-2)
def loss(preds, real):
  return tf.reduce_mean(tf.square(tf.subtract(real, preds)))

In [8]:
@tf.function
def train_step(loss, model, opt, original):
  with tf.GradientTape() as tape:
    preds = model(original)
    error = loss(preds, original)
    gradients = tape.gradient(error, model.trainable_variables)
    vars = zip(gradients, model.trainable_variables)
  opt.apply_gradients(vars)
  return error

In [9]:
def train(model, opt, loss, dataset, epochs=20):
  for epoch in range(1, epochs):
    epoch_loss = 0
    for step, batch_features in enumerate(dataset):
      loss_val = train_step(loss, model, opt, batch_features)
      epoch_loss += loss_val
    model.loss.append(epoch_loss)
    print('Epoch {}/{}. Loss: {}'.format(epoch, epochs, epoch_loss.numpy()))

In [10]:
train(autoencoder, opt, loss, train_data, epochs = epochs)

Epoch 1/50. Loss: 5.159158229827881
Epoch 2/50. Loss: 3.0579447746276855
Epoch 3/50. Loss: 2.952500343322754
Epoch 4/50. Loss: 2.920259952545166
Epoch 5/50. Loss: 2.8625288009643555
Epoch 6/50. Loss: 2.8708505630493164
Epoch 7/50. Loss: 2.840562343597412
Epoch 8/50. Loss: 2.8197474479675293
Epoch 9/50. Loss: 2.835493803024292
Epoch 10/50. Loss: 2.805487632751465
Epoch 11/50. Loss: 2.836609363555908
Epoch 12/50. Loss: 2.817033052444458
Epoch 13/50. Loss: 2.8194468021392822
Epoch 14/50. Loss: 2.8027093410491943
Epoch 15/50. Loss: 2.7878048419952393
Epoch 16/50. Loss: 2.8020923137664795
Epoch 17/50. Loss: 2.8114423751831055
Epoch 18/50. Loss: 2.8193395137786865
Epoch 19/50. Loss: 2.797562837600708
Epoch 20/50. Loss: 2.786923408508301
Epoch 21/50. Loss: 2.8186817169189453
Epoch 22/50. Loss: 2.789909601211548
Epoch 23/50. Loss: 2.8077011108398438
Epoch 24/50. Loss: 2.7880916595458984
Epoch 25/50. Loss: 2.7963504791259766
Epoch 26/50. Loss: 2.8264358043670654
Epoch 27/50. Loss: 2.79987406730